In [17]:
import json
import pandas as pd

In [18]:
import datetime

In [46]:
def prgs_bar(dt0, idx, runs, scale=20):
    import sys

    sys.stdout.write('\r'+str(idx+1) + '/'+str(runs) + ' ' +
                     (' ' if idx+1<runs else '') +
                     " Progress {:2.2%}".format((idx+1) / runs) + ' |' +
                     ''.join(['=']*int(((idx+1) / runs)*scale)) + 
                     ('>' if idx+1<runs else '') +
                     ''.join(['-']*int(scale-int(((idx+1) / runs)*scale)-1)) +
                     '|ETA: ' + str((runs-idx)*(datetime.datetime.now()-dt0)/(idx+1))[:4] + '/' +
                     str((runs)*(datetime.datetime.now()-dt0)/(idx+1))[:4] + 
                    '           ')
    sys.stdout.flush()
    

In [29]:
import os

def add_to_file(new_dict,file_name):
    exists = os.path.isfile(file_name)
    
    if type(new_dict) != list: new_dict = [new_dict]
    
    if exists:
        with open(file_name) as ff:
            json_new = json.load(ff)['key'] 
        json_new = json_new + new_dict
    else:
        json_new = new_dict
    
    json_new2 ={'key':json_new}
    
    with open(file_name, 'w') as f:
        json.dump(json_new2, f, ensure_ascii=False)
        
def load_file(file_name):
    exists = os.path.isfile(file_name)
    if exists:
        with open(file_name) as ff:
            json_new = json.load(ff)['key'] 
    else:
        json_new = []
    
    return json_new

def write_file(new_dict,file_name):
    if type(new_dict) != list: new_dict = [new_dict]
    
    json_new2 ={'key':new_dict}
    
    with open(file_name, 'w') as f:
        json.dump(json_new2, f, ensure_ascii=False)

# Load Ticker Lists

In [51]:
stk_list = 'FTSE'

tick_df=pd.read_csv('./Ticker_TO_KG_ID_Map/'+stk_list+'_Ticker_to_KG.csv',index_col=0)

stk_ticks=list(tick_df.Ticker)
stk_kg_id=list(tick_df.KG_ID)
stk_name=list(tick_df.KG_Search)
tick_df.head()

,Ticker,KG_Search,KG_ID,KG_Desc,Description,Comp_URL,Wiki_URL,KG_nr_hit
0,III.L,3i Group PLC,/m/02ht0r,Private equity company,['16 Palace StreetLONDON \xa0 \xa0 SW1E 5JD\n...,http://www.3i.com/,https://en.wikipedia.org/wiki/3i,0
1,ADML.L,Admiral Group PLC,/m/08vfdz,Insurance company,"['Ty Admiral, David StreetCARDIFF \xa0 \xa0 C...",http://www.admiralgroup.co.uk/,https://en.wikipedia.org/wiki/Admiral_Group,0
2,AAL.L,Anglo American PLC,/m/04vd0h,Mining company,['Anglo American House20 Carlton House Terrace...,http://www.angloamerican.com/,https://en.wikipedia.org/wiki/Anglo_American_plc,0
3,ANTO.L,Antofagasta PLC,/m/05wxdg,NaN,"['Cleveland HSE, 33 King StreetLONDON \xa0 \x...",http://www.antofagasta.co.uk/,https://en.wikipedia.org/wiki/Antofagasta_PLC,0
4,AHT.L,Ashtead Group PLC,/m/08yvj9,Company,['100 CheapsideLONDON \xa0 \xa0 EC2V 6DT\nP: ...,http://www.ashtead-group.com/,https://en.wikipedia.org/wiki/Ashtead_Group,0


# Set file lists

In [31]:
path='./entities/'

list_files=['rss_crawl2019-04-26 04-03-25_entities.txt',
       'rss_crawl2019-04-25 23-00-26_entities.txt', 
       'rss_crawl2019-04-10 08-45-38_entities.txt',
       'rss_crawl2019-04-13 02-07-56_entities.txt',
       'rss_crawl2019-04-23 07-51-54_entities.txt',
       'rss_crawl2019-04-20 09-10-00_entities.txt',
      ]

In [32]:
for r, d, f in os.walk(path):
    list_files=f

list_files=list(set(list_files)-{'.DS_Store'})
len(list_files)

91

# Build Stock Ticker Files

The loop reuns through each News file and allocates any news that match a KG ID from a provided list of KG_IDs and a list of files to go through. Any hit against a KG_ID gets written to 
 1. A file with the full news article in the same format
 2. An Index file ending in _idx_ which stores the ids of all matching articles

INPUTS:
 1. stk_kg_id - List of KG_IDs to search
 2. stk_ticks - List of stock tickers
 3. list_files - list of files to search through
 3. salience_threshold - Minimal relevancy of the the stock to an article
 
BENEFITS:

This setup has a few useful benefits
 1. Does not rely on keeping track - the index files keep track for you of which news were included so even if a file has been run through twice the news will only exist once
 2. Can be flexible in changing the list of stocks or list of news files
 3. One can use this for a daily update and a big batch update where the day's file is just dumped here and allocated 

## An Improved Design

The current design runs through each news file and for each hit loads the stock and the index file, saves the hit and moves on. This is not efficient as loading and closing the files takes a long time, especially when stock files start to get bigger.

Improved Design: 
 1. Change the index file to be a central index file which is a dictionary by stock ticker and holds all news IDs by Stock ticker to look up presence
 2. Per each news file, a small dictionary by stock ticker with matching content from the current file will be added to. 
 3. When done with a file, the dictionary will allocate all results from the file to news files at one go and store the new version of the index file. News distionary will be reset for the next file

In [52]:
salience_threshold = 0.005
path_store = './stk_news_list/'


nr_hits = 0
ttl_hits = 0

# Create empty dictionaries for indices and news per file
# Index dictionary will be loaded with all existing indices
dict_idx={}
for stk in stk_ticks:
    idx_file = load_file(path_store+stk+'_idx.txt')
    if len(idx_file) > 0: dict_idx[stk] = idx_file
dict_nws={}

for file in list_files:
    # Open news file
    with open(path+file) as ff:
        json2 = json.load(ff)['key']
    print('\nLoaded: ' + file)
    
    dt0 = datetime.datetime.now()
    

    
    
    for idx in range(len(json2)):
        if len(json2[idx].get('entities',''))>0:
            kg_ids_art=[json2[idx]['entities'][loop]['mid'] 
                        for loop in range(len(json2[idx]['entities']))
                        if json2[idx]['entities'][loop].get('salience',0)>salience_threshold]

            stk_news_intersect = list(set(stk_kg_id) & set(kg_ids_art))
            for stk_kg_itrs in stk_news_intersect:
                #print(stk_kg_itrs)
                loc_in_list = stk_kg_id.index(stk_kg_itrs)
                stk_ticker = stk_ticks[loc_in_list]
                

                if json2[idx]['id'] not in dict_idx.get(stk_ticker,[]):
                    dict_nws[stk_ticker]=dict_nws.get(stk_ticker,[]) + [json2[idx]]
                    dict_idx[stk_ticker]=dict_idx.get(stk_ticker,[]) + [json2[idx]['id']]

                nr_hits = nr_hits +1
        prgs_bar(dt0, idx, len(json2), scale=20)
        ttl_hits = ttl_hits + 1
    
# Save down updated files at the end of the round
print('Saving...')
for stk in stk_ticks:
    #assert(len(dict_idx.get(stk,[])) == len(dict_nws.get(stk,[])))
    if len(dict_nws.get(stk,[]))>0:
        add_to_file(dict_nws[stk],path_store+stk+'.txt')
        write_file(dict_idx[stk],path_store+stk+'_idx.txt')


Loaded: rss_crawl2019-04-09 00-24-58.063131_entities.txt
100/100  Progress 100.00% |====================|ETA: 0:00/0:00           
Loaded: rss_crawl2019-04-13 18-00-18_entities.txt
10000/10000  Progress 100.00% |====================|ETA: 0:00/0:00           
Loaded: rss_crawl2019-04-25 23-00-26_entities.txt
8450/8450  Progress 100.00% |====================|ETA: 0:00/0:00           
Loaded: rss_crawl2019-04-05 13-09-32_entities.txt
43/43  Progress 100.00% |====================|ETA: 0:00/0:00           
Loaded: rss_crawl2019-04-05 22-30-16_entities.txt
294/294  Progress 100.00% |====================|ETA: 0:00/0:00           
Loaded: rss_crawl2019-04-12 16-23-53_entities.txt
3600/3600  Progress 100.00% |====================|ETA: 0:00/0:00           
Loaded: rss_crawl2019-04-04 231136018255_entities.txt
134/134  Progress 100.00% |====================|ETA: 0:00/0:00           
Loaded: rss_crawl2019-04-11 22-28-45_entities.txt
755/839   Progress 89.99% |=================>--|ETA: 0:00/0:00  

410/410  Progress 100.00% |====================|ETA: 0:00/0:00           
Loaded: rss_crawl2019-04-05 12-31-30_entities.txt
177/177  Progress 100.00% |====================|ETA: 0:00/0:00           
Loaded: rss_crawl2019-04-19 18-11-00_entities.txt
10000/10000  Progress 100.00% |====================|ETA: 0:00/0:00           
Loaded: rss_crawl2019-04-24 18-09-52_entities.txt
5200/5200  Progress 100.00% |====================|ETA: 0:00/0:00           
Loaded: rss_crawl2019-04-23 17-04-16_entities.txt
7450/7450  Progress 100.00% |====================|ETA: 0:00/0:00           
Loaded: rss_crawl2019-04-21 21-08-57_entities.txt
6400/6400  Progress 100.00% |====================|ETA: 0:00/0:00           
Loaded: rss_crawl2019-04-23 07-51-54_entities.txt
10000/10000  Progress 100.00% |====================|ETA: 0:00/0:00           
Loaded: rss_crawl2019-04-03 232220050155_entities.txt
118/118  Progress 100.00% |====================|ETA: 0:00/0:00           
Loaded: rss_crawl2019-03-25 133527.71708

In [53]:
# SnP: 42612 / 394418
# FTSE: 10296 / 394418

print(str(nr_hits) + ' / ' + str(ttl_hits))

10296 / 394418
